# Install

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [6]:
import sys
sys.path.append('drive/MyDrive/MemeProject/src/sbert/preprocessing')
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', 'drive/MyDrive/MemeProject/src/sbert/preprocessing']


In [7]:
from tqdm import tqdm
import config as cfg
import pandas as pd
import swifter
from pshmodule.utils import filemanager as fm

In [8]:
# df1 = fm.load(cfg.data_path_v1)
# df2 = fm.load(cfg.data_path_v2)
df = fm.load(cfg.data_path_final)

extension : .xlsx
Loaded 123696 records from drive/MyDrive/MemeProject/data/row_data/final/final.xlsx


In [9]:
df.head()

,차수,관리번호,밈 id,set,검수,utt,발화구분,대화소재,ARG0,발화문,ARG1,ARG_op,서술어,화행(U),meme_ID,meme
0,1,Aa0001,DT0044,1.0,1.0,1,U,학교생활,나,나 중간고사 반에서 1등했어,중간고사,NaN,반에서 1등하다,자랑,NaN,NaN
1,1,NaN,NaN,NaN,NaN,2,U,학교생활,나,중간고사 점수 내가 반에서 제일 잘 받음,중간고사,NaN,점수 반에서 제일 잘 받다,자랑,NaN,NaN
2,1,NaN,NaN,NaN,NaN,3,U,학교생활,나,나 반에서 중간 성적 제일 좋아,중간,NaN,반에서 성적 제일 좋다,자랑,NaN,NaN
3,1,NaN,NaN,NaN,NaN,4,U,학교생활,나,우리 반에서 내가 시험 제일 잘 봤다,시험,NaN,우리 반에서 제일 잘 보다,자랑,NaN,NaN
4,1,NaN,NaN,NaN,NaN,5,U,학교생활,NaN,중간고사 반 1등 먹음,중간고사,NaN,반 1등 먹다,자랑,NaN,NaN


In [ ]:
# new_header = df1.iloc[1]
# df1 = df1[2:]
# df1.columns = new_header

# Reshape

#### Merge

In [10]:
# df1 = df1[['관리번호', 'utt',	'발화구분',	'대화소재', '화행(U)', '발화문',	'ARG1',	'ARG_op']]
# df2 = df2[['관리번호', 'utt',	'발화구분',	'대화소재', '화행(U)', '발화문',	'ARG1',	'ARG_op']]
df = df[['관리번호', 'utt',	'발화구분',	'대화소재', '화행(U)', '발화문',	'ARG1',	'ARG_op']]

In [11]:
df.head()

,관리번호,utt,발화구분,대화소재,화행(U),발화문,ARG1,ARG_op
0,Aa0001,1,U,학교생활,자랑,나 중간고사 반에서 1등했어,중간고사,NaN
1,NaN,2,U,학교생활,자랑,중간고사 점수 내가 반에서 제일 잘 받음,중간고사,NaN
2,NaN,3,U,학교생활,자랑,나 반에서 중간 성적 제일 좋아,중간,NaN
3,NaN,4,U,학교생활,자랑,우리 반에서 내가 시험 제일 잘 봤다,시험,NaN
4,NaN,5,U,학교생활,자랑,중간고사 반 1등 먹음,중간고사,NaN


In [12]:
print(len(df))

123696


In [13]:
# df_A = df2[df2['관리번호'].str.startswith('A')]
# df_E = df2[df2['관리번호'].str.startswith('E')]

In [14]:
# df = pd.concat([df_A, df1, df_E])
df.reset_index(inplace=True, drop=True)

In [15]:
df.shape

(123696, 8)

#### Numbering

In [16]:
# no
num_list = [i+1 for i in range(len(df))]
df['num'] = num_list

# group no
g_list = []
num = 1
for i in range(len(df)):
  if i % 24 == 0 and i != 0:
    num += 1
  g_list.append(num)

df['g_num'] = g_list

In [17]:
df = df[['num', 'g_num', 'utt', '발화구분',	'대화소재', '화행(U)', '발화문', 'ARG1', 'ARG_op']]
df.rename(columns={'발화구분':'type', '발화문':'u',	'대화소재':'material', '화행(U)':'speech', 'ARG1':'arg1', 'ARG_op':'arg_op'}, inplace=True)
df = df.fillna('')

<ipython-input-17-e19f0d1245ce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'발화구분':'type', '발화문':'u',	'대화소재':'material', '화행(U)':'speech', 'ARG1':'arg1', 'ARG_op':'arg_op'}, inplace=True)


In [18]:
df.head()

,num,g_num,utt,type,material,speech,u,arg1,arg_op
0,1,1,1,U,학교생활,자랑,나 중간고사 반에서 1등했어,중간고사,
1,2,1,2,U,학교생활,자랑,중간고사 점수 내가 반에서 제일 잘 받음,중간고사,
2,3,1,3,U,학교생활,자랑,나 반에서 중간 성적 제일 좋아,중간,
3,4,1,4,U,학교생활,자랑,우리 반에서 내가 시험 제일 잘 봤다,시험,
4,5,1,5,U,학교생활,자랑,중간고사 반 1등 먹음,중간고사,


In [19]:
df.tail()

,num,g_num,utt,type,material,speech,u,arg1,arg_op
123691,123692,5154,20,CPO,,,,,
123692,123693,5154,21,CPO,,,,,
123693,123694,5154,22,CPO,,,,,
123694,123695,5154,23,CPO,,,,,
123695,123696,5154,24,CPO,,,,,


#### Preprocessing

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123696 entries, 0 to 123695
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   num       123696 non-null  int64 
 1   g_num     123696 non-null  int64 
 2   utt       123696 non-null  int64 
 3   type      123696 non-null  object
 4   material  123696 non-null  object
 5   speech    123696 non-null  object
 6   u         123696 non-null  object
 7   arg1      123696 non-null  object
 8   arg_op    123696 non-null  object
dtypes: int64(3), object(6)
memory usage: 8.5+ MB


#### Reshape

In [21]:
result = []
temp = []

for i in tqdm(df.iterrows()):
  temp.append([i[1]['num'], i[1]['g_num'], i[1]['utt'], i[1]['type'], i[1]['u'], i[1]['arg1'], i[1]['arg_op'], i[1]['material'], i[1]['speech']])
  
  if i[1]['num'] % 24 == 0 and i[1]['num'] != 0:
    temp_u = temp[:6]
    temp_cpr = temp[6:12]
    temp_mm = temp[12:18]
    temp_cpo = temp[18:24]

    for u in temp_u:
      # 밈 조합
      for cpr in temp_cpr:
        for mm in temp_mm:
          for cpo in temp_cpo:
            meme = "{} {} {}".format(cpr[4], mm[4], cpo[4]).strip()
            result.append([u[0], u[1], u[2], u[7], u[8], u[4], meme, u[5], u[6]])
    # 24개 씩 초기화
    temp = []

123696it [00:28, 4304.50it/s]


In [22]:
len(result)

6679584

In [23]:
df = pd.DataFrame(result, columns=[['num', 'g_num', 'u_num', 'material', 'speech', 'u', 'meme', 'arg1', 'arg_op']])

In [24]:
df = df[['u', 'meme']]

In [25]:
df.drop_duplicates(inplace=True)

In [30]:
df.reset_index(inplace=True, drop=True)

In [26]:
num_list = [i for i in range(len(df))]
df['num'] = num_list

In [27]:
df = df[['num', 'u', 'meme']]

In [31]:
df.tail()

,num,u,meme
528401,528401,내일운 지오디 10주년 콘섵 관람합니댜,잘 가 (가지 마) 행복해 (떠나지 마) 나를 잊어줘 잊고 살아가 줘 (나를 잊지 마)
528402,528402,낼 god 10주년 기념 콘 조집니다,다같이 이거 부르겠네? 잘 가 (가지 마) 행복해 (떠나지 마) 나를 잊어줘 잊고 ...
528403,528403,낼 god 10주년 기념 콘 조집니다,아마 이 노래 부르겠지? 잘 가 (가지 마) 행복해 (떠나지 마) 나를 잊어줘 잊고...
528404,528404,낼 god 10주년 기념 콘 조집니다,이거 부를거임? 잘 가 (가지 마) 행복해 (떠나지 마) 나를 잊어줘 잊고 살아가 ...
528405,528405,낼 god 10주년 기념 콘 조집니다,잘 가 (가지 마) 행복해 (떠나지 마) 나를 잊어줘 잊고 살아가 줘 (나를 잊지 마)


In [32]:
df.shape

(528406, 3)

# Save

In [33]:
import json

In [34]:
temp_dict = [{"num": row['num'], "u": row['u'].strip(), "meme": row['meme'].strip()} for _, row in df.iterrows()]
with open(cfg.ref_data, 'w', encoding='utf-8') as f:
    for line in temp_dict:
        json_record = json.dumps(line, ensure_ascii=False)
        f.write(json_record + '\n')